## 5. Análise Exploratória dos Dados

In [ ]:
import pandas as pd

data_folder = "../data/cleaned_data.csv"

# Carregando dados limpos (resultado da Pré-Tokenização)
df_enem = pd.read_csv(data_folder)
df_enem.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

df = df_enem.rename(columns={"nu_param_B": "dificuldade"})
df.head()

# Análise Preliminar

Dado que `nu_param_B` (aka. `dificuldade`, por aqui) é a variável target, desnecessário focar em questões em que esta não foi calculada

In [ ]:
df.isna().sum()

In [138]:
df = df[df['dificuldade'].notna()]

In [ ]:
df.describe()

## **Análise da Dificuldade**

In [ ]:
sns.histplot(df['dificuldade'], bins=30, kde=True)
plt.title('Distribuição da Dificuldade')
plt.xlabel('Dificuldade')
plt.ylabel('Frequência')

In [ ]:
stats.probplot(df['dificuldade'], dist="norm", plot=plt)
plt.title('Q-Q Plot (Dificuldade)')
plt.show()

In [ ]:
stat, p = stats.shapiro(df[df['dificuldade'].notna()]['dificuldade'])
print(f"Estatística de teste: {stat:.4f} | p-valor: {p:.4f}")

if p > 0.05:
    print("Não rejeita H_0: distribuição normal.")
else:
    print("Rejeita H_0: exclui distribuição normal.")


Com a exclusão da distribuição normal, necessário o grupo se atentar se o modelo de predição possui sensibilidade à distribuição. 

Caso seja, possível aplicar Box-Cox.

In [ ]:
sns.boxplot(x='ano', y='dificuldade', data=df, order=sorted(df["ano"].unique()))
plt.title('Dificuldade por Ano')
plt.xticks(rotation=45)

plt.show()

In [ ]:
def max_sem_outlier(group):
    Q1 = group.quantile(0.25)
    Q3 = group.quantile(0.75)
    IQR = Q3 - Q1
    limite_superior = Q3 + 1.5 * IQR
    return group[group <= limite_superior].max()  

max_por_ano = df.groupby('ano')['dificuldade'].apply(max_sem_outlier)
max_por_ano

O corte de dificuldade 3, feito no artigo, não parece de todo arbitrário.

Afinal, na maioria dos anos, o máximo de dificuldade (excluídos os outliers) não ultrapassa 3 frequentemente. Como sugestão, possível utilizar o corte do máximo da lista (ano de 2013, com 3.38570)

# **Análise enunciados**

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
df['tam_enunciado'] = df['enunciado'].str.len()
df['tam_enunciado_limpo'] = df['enunciado_limpo'].str.len()

sns.histplot(df['tam_enunciado'], bins=30, kde=True, ax=axes[0])
axes[0].set_title('Distribuição do Tamanho dos Enunciados')

sns.histplot(df['tam_enunciado_limpo'], bins=30, kde=True, ax=axes[1])
axes[1].set_title('Distribuição do Tamanho dos Enunciados (sem stop word)')

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
sns.scatterplot(x='tam_enunciado', y='dificuldade', data=df, ax=axes[0])
axes[0].set_title('Tamanho do Enunciado vs Dificuldade')

sns.scatterplot(x='tam_enunciado_limpo', y='dificuldade', data=df, ax=axes[1])
axes[1].set_title('Tamanho do Enunciado vs Dificuldade (sem stop word)')

x_min = df['tam_enunciado_limpo'].min()
x_max = df['tam_enunciado'].max()
axes[0].set_xlim(x_min, x_max)
axes[1].set_xlim(x_min, x_max)

plt.tight_layout()
plt.show()

Não se verifica um correlação direta entre o tamanho do enunciado e a dificuldade da questão, muito embora se verifique uma pequena concentração entre 0 e 2 nos menores enunciados.

## **Análise de Gabarito**

In [ ]:
df['gabarito'].value_counts().plot(kind='bar', title='Distribuição de Alternativas Corretas')

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
df['tam_alternativas'] = df['alternativas'].str.len()
df['tam_alternativas_limpo'] = df['alternativas_limpo'].str.len()

sns.histplot(df['tam_alternativas'], bins=30, kde=True, ax=axes[0])
axes[0].set_title('Distribuição do Tamanho das Alternativass')

sns.histplot(df['tam_alternativas_limpo'], bins=30, kde=True, ax=axes[1])
axes[1].set_title('Distribuição do Tamanho das Alternativass (sem stop word)')

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
sns.scatterplot(x='tam_alternativas', y='dificuldade', data=df, ax=axes[0])
axes[0].set_title('Tamanho das Alternativas vs Dificuldade')

sns.scatterplot(x='tam_alternativas_limpo', y='dificuldade', data=df, ax=axes[1])
axes[1].set_title('Tamanho das Alternativas vs Dificuldade (sem stop word)')

x_min = df['tam_alternativas_limpo'].min()
x_max = df['tam_alternativas'].max()
axes[0].set_xlim(x_min, x_max)
axes[1].set_xlim(x_min, x_max)

plt.tight_layout()
plt.show()

# WordCloud (Gabarito e Alternativas)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

In [154]:
def get_wordcloud(df, col):
    # 1. Filtrar os dados
    df_counter = df.dropna(subset=[col])  # Remove NaN
    median_dif = df_counter['dificuldade'].median()

    # 2. Textos para cada grupo
    text_total = ' '.join(df_counter[col])
    text_hard = ' '.join(df_counter[df_counter['dificuldade'] > median_dif][col])
    text_easy = ' '.join(df_counter[df_counter['dificuldade'] <= median_dif][col])

    # 3. Configurar as nuvens
    wordcloud_total = WordCloud(width=600, height=300, background_color='white').generate(text_total)
    wordcloud_hard = WordCloud(width=600, height=300, background_color='white').generate(text_hard)
    wordcloud_easy = WordCloud(width=600, height=300, background_color='white').generate(text_easy)

    # 4. Plotar em uma linha
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))

    # Nuvem 1: Total
    axes[0].imshow(wordcloud_total, interpolation='bilinear')
    axes[0].set_title('Todos os Enunciados', fontsize=12)
    axes[0].axis('off')

    # Nuvem 2: Maior dificuldade
    axes[1].imshow(wordcloud_hard, interpolation='bilinear')
    axes[1].set_title(f'Maior Dificuldade ( > {median_dif:.2f})', fontsize=12)
    axes[1].axis('off')

    # Nuvem 3: Menor dificuldade
    axes[2].imshow(wordcloud_easy, interpolation='bilinear')
    axes[2].set_title(f'Menor Dificuldade ( ≤ {median_dif:.2f})', fontsize=12)
    axes[2].axis('off')

    plt.tight_layout()
    plt.show()

In [156]:
def get_word_frequencies(df, col, top_n = 10):
    # 1. Filtrar os dados
    df_counter = df.dropna(subset=[col])
    median_dif = df_counter['dificuldade'].median()

    # 2. Definir os grupos
    text_total = df_counter[col]
    text_hard = df_counter[df_counter['dificuldade'] > median_dif][col]
    text_easy = df_counter[df_counter['dificuldade'] <= median_dif][col]

    # 3. Contar palavras (top 10 em cada grupo)
    def get_top_words(text_series, n=top_n):
        vectorizer = CountVectorizer()
        X = vectorizer.fit_transform(text_series)
        word_counts = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out()).sum()
        return word_counts.sort_values(ascending=False).head(n)

    top_total = get_top_words(text_total)
    top_hard = get_top_words(text_hard)
    top_easy = get_top_words(text_easy)

    # 4. Plotar histogramas lado a lado
    fig, axes = plt.subplots(1, 3, figsize=(18, 6), sharey=True)

    # Histograma 1: Total
    axes[0].barh(top_total.index, top_total.values, color='skyblue')
    axes[0].set_title('Todos os Enunciados')
    axes[0].invert_yaxis()

    # Histograma 2: Alta dificuldade
    axes[1].barh(top_hard.index, top_hard.values, color='salmon')
    axes[1].set_title(f'Maior Dificuldade ( > {median_dif:.2f})')

    # Histograma 3: Baixa dificuldade
    axes[2].barh(top_easy.index, top_easy.values, color='lightgreen')
    axes[2].set_title(f'Menor Dificuldade ( ≤ {median_dif:.2f})')

    plt.tight_layout()
    plt.show()

In [ ]:
get_wordcloud(df, "enunciado_limpo")

In [ ]:
get_word_frequencies(df, "enunciado_limpo")

In [ ]:
df['alternativas_adap'] = df['alternativas_limpo'].dropna().str.replace(r'[A-E]:\s*', '', regex=True)

get_wordcloud(df, "alternativas_adap")

In [ ]:
get_word_frequencies(df, "alternativas_adap")